# Math 134A Week 4

In [7]:
import math
from scipy.optimize import fsolve
import numpy_financial as npf
import pandas as pd

# Computing the present value of a cash flow with interest rate r. 
def PV(r, cf, continuous_compounding = False, dt = None): 
    if dt is None: dt = range(len(cf))
    pv = 0
    if not continuous_compounding:
        for k in range(0, len(cf)):
            pv += cf[k] / (1+r)**dt[k]
    else:
        for k in range(0, len(cf)):
            pv += cf[k] * math.exp(-r * dt[k])
    return pv

# This will return the future value of a cash flow at time T. If T is not specified, then we take T as the time that we receive the last cash flow
def FV(r, cf, continuous_compounding = False, dt = None, T = None):
    if dt is None: dt = range(len(cf))
    if T is None: T = dt[-1]
    fv = 0
    if not continuous_compounding:
        for k in range(0, len(cf)):
            fv += cf[k] * (1+r)**(T - dt[k])
    else:
        for k in range(0, len(cf)):
            fv += cf[k] * math.exp(r * (T - dt[k]))
    return fv   

# Computing the internal rate of return by using the fsolve function to find roots to a polynomial
def irr(cf):
    def simple_npv(r, cf):
        return sum(cf[k]/(1+r)**k for k in range(len(cf)))
    
    result = fsolve(simple_npv, x0 = 0.1, args = cf)
    return result[0]

# Computing the internal rate of return by using the irr function in numpy financial library
def irr2(cf):
    return npf.irr(cf)

# This function computes the YTM of a bond mature in dt years, with face value F, price P, and coupon c paid m times every year
def YTM(F, P, dt, c = 0, m = 1):
    cf = [-P] + [c/m] * (dt * m - 1) + [F+c/m]
    return irr(cf) * m
    
# This function computes the price of a bond mature in dt years, with face value F, YTM y, and yearly coupon c paid m times every year 
def bond_price(F, y, dt, c = 0, m = 1):
    cf = [0] + [c/m] * (dt * m - 1) + [F+c/m]
    return PV(y/m, cf)

# This function will return the per period payment A if P is loaned for n periods, with interest r per period.
def A(P, r, n):
    return (P * r)/(1 - 1/(1+r)**n)

def macaulay_duration(c, y, m, n):
    """
    c is the coupon rate per period
    y is the yield per period
    m is the number of periods per year
    n is the number of periods remaining
    """
    D = (1+y)/(m*y) - (1 + y + n*(c-y))/(m*c * ((1+y)**n - 1) + m*y)
    return D

# Duration of a Portfolio
Assume that a portfolio consists of two bonds with payment cash flow $(0, c_1, c_2,\ldots, c_n)$ and $(0, d_1,\ldots, d_n)$. We may extend one of the cash flow with zeros so that they have the same length. We want to compute the duration of this portfolio. Let $P^A_k$ be the present value of the payment $c_k$, and $P^B_k$ be the resent value of the payment $d_k$. Let $P^A$ be the price/present value of the first bond, and $P^B$ be the price of the second bond. Let $D^A$ and $D^B$ to be their durations. Then we have 
$$D^A = \frac{\sum_1^n t_kP^A_k}{P^A}$$
$$D^B = \frac{\sum_1^n t_kP^B_k}{P^B}$$

Let $D$ be the duration of this portfolio, and $P = P_A+P_B$ to be the present value of this portfolio. Then we have 
$$D = \frac{\sum_1^n t_k (P^A_k+P^B_k)}{P} = \frac{D^AP^A+D^BP^B}{P} = \frac{P^A}{P}D^A + \frac{P^B}{P}D^B$$

In other words, the duration of a portfolio is a weighted sum of the duration of each individual bonds, and the weight is the proportion of this bonds price over the price of this portfolio.

More generally, if a portfolio consists of $n$ fixed income investments with price $P_1,\ldots, P_n$, and each of them has a duration $D_1,\ldots, D_n$. Let $P = P_1+\cdots+P_n$ be the price of this portfolio, and $D$ be its duration. Then we have 
$$D = \frac{P_1}{P}D_1+\cdots+\frac{P_n}{P}D$$

# Immunization
The goal of **Immunization** is to protect a portfolio against the change in the interest rate. If we have an obligation of paying $F$ amount of money in $D$ years. Suppose that the prevailing interest is $r$. Then the YTM of a bond will be close to $r$. A zero coupon bond with face value $F$ and mature in $D$ years will serve the purpose. However, it is very common that there is no such a bond in the market. We may construct a portfolio of other bonds so that the present value of this portfolio will equal to the present value of the future payment. However, if the interest rate change, then our portfolio may not worth the same as the payment that we need to deliver. 

Then what we do is to construct a portfolio consisting of fixed income securities so that its present value matches with the present value of the future payment, its duration matches with $D$, which is the time we need to deliver the payment, or equivalently, the duration of the hypothetical zero-coupon bond which mature in $D$ years. In this case, if the interest rate change, then the present value of the future payment and the present value of our portfolio will have the same amount of change. 

**Exercise 3.16:** (Bond selection) 
Consider the four bonds having annual payments as shown in the following table. They are traded to produce a 15\% yield.
1. Determine the price of each bond.
2. Determine the duration of each bond (not the modified duration).
3. Which bond is most sensitive to a change in yield?
4. Suppose you owe 2,000 at the end of 2 years. Concern about interest rate risk suggests that a portfolio consisting of the bonds and the obligation should be immunized. If
$V_A, V_B, V_C$, and $V_D$ are the total values of bonds purchased of types A,B,C, and D, respectively, what are the necessary constraints to implement the immunization?
5. In order to immunize the portfolio, you decide to use bond C and one other bond. Which other bond should you choose? Find the amounts (in total value) of each of these to purchase.
6. You decided in 5 to use bond C in the immunization. Would other choices, including perhaps a combination of bonds, lead to lower total cost?

<center>

|        | Bond A | Bond B | Bond C | Bond D |
|--------|--------|--------|--------|--------|
| Year 1 | 100    | 50     | 0      | 1000   |
| Year 2 | 100    | 50     | 0      | 0      |
| Year 3 | 1100   | 1050   | 1000   | 0      |

</center>

**Exercise 3.6b)**:
Consider an infinite cash flow $(1, 2, 3, 4, 5,\ldots)$. Suppose that the interest rate is $r$ and define $x = \frac{1}{1+r}$. Find the present value of this cash flow as a function of $r$ or $x$. 

**Exercise 3.10:** (Variable-rate mortgage)
The Smith family just took out a variable-rate mortgage on their new home. The mortgage value is 100,000, the term is 30 years, and initially the interest rate is 8\%. The interest rate is guaranteed for 5 years, after which time the rate will be adjusted according to prevailing rates. The new rate can be applied to their loan either by changing the payment amount or by changing the length of the mortgage.

(a) What is the original yearly mortgage payment? (Assume payments are yearly.)\
(b) What will be the mortgage balance after 5 years?\
(c) If the interest rate on the mortgage changes to 9% after 5 years, what will be the new yearly payment that keeps the termination time the same?\
(d) Under the interest change in (c), what will be the new term if the payments remain the same?

# Annual Worth
Given a cash flow $(x_0,\ldots, x_n)$. The **annual worth** of this cashflow is the number $A$ such that $(0, A, \ldots, A)$ (ends in the n-th period) is an equivalent cashflow. The annual worth is useful for comparing investments with different time cycles.

**Exercise 3.2**: (Cycles and annual worth)
Given a cash flow stream $X = (x_0,x_1,x_2,...,x_n)$, a new stream $X_\infty$ of infinite length is made by successively repeating the corresponding finite stream. The interest rate is r. Let P and A be the present value and the annual worth, respectively, of stream X. Finally, let $P_\infty$ be the present value of stream $X_\infty$. Find A in terms of $P_\infty$ and conclude that $A$ can be used as well as $P_\infty$ for evaluation purposes.

**Example**: (Conflicting Recommendations)
Consider the following investments whose cashflows are given by 
$X_A = (-1, 2)$\
$X_B = (-1, 0, 3)$\
Suppose that the prevailing interest is 10%. 

a) Compute their present values, IRR \
b) If both of them are one-time opportunity, which investment would you choose? \
c) If both of the investments can be repeated, which investment would you choose? \
d) Compute their annual worth and $P_\infty$. \
e) Repeat c) with argument by the annual worth or $P_\infty$.

In [19]:
cf_A = [-1, 2]
cf_B = [-1, 0, 3]
r = 0.1
PV_a = PV(r, cf_A)
PV_b = PV(r, cf_B)
r_a = irr(cf_A)
r_b = irr(cf_B)
A_a = A(PV_a, r, len(cf_A) - 1)
A_b = A(PV_b, r, len(cf_B) - 1)
P_infty_a = PV_a/(1-(1/(1+r))**(len(cf_A) - 1))
P_infty_b = PV_b/(1-(1/(1+r))**(len(cf_B) - 1))
print(f"a) The present value of A is {PV_a:.2f}. Its irr is {r_a * 100:.2f}%")
print(f"   The present value of B is {PV_b:.2f}. Its irr is {r_b * 100:.2f}%")

a) The present value of A is 0.82. Its irr is 100.00%
   The present value of B is 1.48. Its irr is 73.21%
